### Inspect Expert Output

This notebook shows how to visualize results of expert after running it locally.

##### Steps

Follow this [guide](https://ln2697.github.io/lead/docs/data_collection.html) to produce some local data first.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%load_ext ipython_beartype
%beartype

import cv2
import matplotlib.pyplot as plt
import torch

from lead.common.visualizer import Visualizer
from lead.data_loader.carla_dataset import CARLAData
from lead.training.config_training import TrainingConfig

Setup dataloader and point the root to the output directory

In [ ]:
training_config = TrainingConfig()
training_config.visualize_dataset = True
training_config.use_sensor_perburtation = False

training_config.carla_root = "data/expert_debug"

dataset = CARLAData(root=training_config.carla_data, config=training_config, random=False)

Visualize one sample

In [ ]:
for index in range(len(dataset)):  # You can change this index to test different samples
    sample = dataset[index]
    sample = torch.utils.data._utils.collate.default_collate([sample])  # Collate into a batch of size 1

    visualizer = Visualizer(config=training_config, data=sample, prediction=None)
    image = visualizer.visualize_training_labels()
    stacked_perspective = visualizer.stacked_perspectives
    stacked_perspective = cv2.cvtColor(stacked_perspective, cv2.COLOR_RGB2BGR)
    cv2.imwrite(f"outputs/visualization/stacked_perspectives_{index:04d}.png", stacked_perspective)
    plt.figure(figsize=(20, 10))
    plt.imshow(image)
    plt.show()

Create an animated WebP from the generated images

In [ ]:
from pathlib import Path
from PIL import Image

def create_webp_gif(image_dir, output_path, duration=500):
    """
    Create an animated WebP from images, using only the left half of each image.
    
    Args:
        image_dir: Directory containing the images
        output_path: Path to save the output WebP file
        duration: Duration of each frame in milliseconds
    """
    image_dir = Path(image_dir)
    image_files = sorted(image_dir.glob("stacked_perspectives_*.png"))
    
    if not image_files:
        print(f"No images found in {image_dir}")
        return
    
    frames = []
    for img_path in image_files[200:500]:  # Limit to first 500 images
        img = Image.open(img_path)
        # Take only the left half
        width, height = img.size
        left_half = img.crop((0, 0, width // 2, height))
        frames.append(left_half)
    
    if frames:
        # Save as animated WebP
        frames[0].save(
            output_path,
            save_all=True,
            append_images=frames[1:],
            duration=duration,
            loop=0,
            compression=9,
            quality=60
        )
        print(f"Created animated WebP with {len(frames)} frames: {output_path}")
    
# Create the animated WebP
create_webp_gif("outputs/visualization", "outputs/visualization.webp", duration=50)